### Importação das bibliotecas necessárias para a raspagem de dados.

In [ ]:
print('oi')

In [ ]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from time import sleep
import requests
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())

options = Options()
options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.

### Acessando o site Open Street Map e capturando as rotas pendentes e as rotas finalizadas.

In [ ]:
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

In [ ]:
#list_finished_routes
#list_pending_routes

### Acessando a telas para efetuar os downloads.

In [ ]:
## Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
navegador = webdriver.Chrome(service=servico)
for list_route in list_routes:
    sleep(5)
    url = list_route[0]
    navegador.get(url)  #--> Exemplo onde usamos o Selenium somente com o [.get]
    navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
navegador.close()

### Executando de uma só vez

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from time import sleep
import requests
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())

options = Options()
options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
#options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.

# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
#navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
#navegador = webdriver.Chrome(service=servico)
for list_route in list_routes:
    sleep(3)
    url = list_route[0]
    navegador.get(url)  #--> Exemplo onde usamos o Selenium somente com o [.get]
    navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
navegador.close()

### Movendo os arquivos da pasta Download para o MINIO_DATA_LAKE

In [ ]:
# pip install minio
import os
from minio import Minio
from minio.error import S3Error
DOWNLOADS = '/home/thiago/Downloads/'
DATALAKE_STAGE = 'DATALAKE_STAGE'  # Substitua pelo caminho do diretório de destino
MINIO_DATA_LAKE = 'datalake'

In [ ]:
minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)
#print(minioclient.list_buckets())
#print("Quantidade de Buckets", len(minioclient.list_buckets()))

In [ ]:
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for file_name in files_to_rename:
        new_name = file_name.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, file_name), os.path.join(DOWNLOADS, new_name))

In [ ]:
## Aqui os arquivos com a extensão .gpx saem da pasta Download para o bucket MinIO
files_to_move = [files for files in os.listdir(DOWNLOADS) if files.endswith(".gpx")]
for name_file in files_to_move:
    local_path = os.path.join(DOWNLOADS, name_file)    
    if os.path.isfile(local_path):
        try:
            minioclient.fput_object(MINIO_DATA_LAKE, name_file, local_path)
            print(f"Arquivo {name_file} enviado com sucesso para o bucket.")
        except S3Error as e:
            print(f"Erro ao enviar o arquivo: {item} -> Erro: {e}")

In [ ]:
## Aqui os arquivos saem da pasta download para a pasta DATALAKE_STAGE

# files_to_move = [files for files in os.listdir(DOWNLOADS) if files.endswith(".gpx")]
# for name_files in files_to_move:
#     origin_path = os.path.join(DOWNLOADS, name_files)
#     destiny_path = os.path.join(DATALAKE_STAGE, name_files)
#     os.rename(origin_path, destiny_path)
#     print(f"Arquivo {name_files} movido para {destiny_path}")

In [ ]:
## Aqui os arquivos saem da pasta DATALAKE_STAGE para o bucket MinIO

# for item in os.listdir(DATALAKE_STAGE):
#     local_path = os.path.join(DATALAKE_STAGE, item)
#     print(local_path)
# 
#     if os.path.isfile(local_path):
#         try:
#             minioclient.fput_object(DATA_LAKE, item, local_path)
#             print(f"Arquivo {item} enviado com sucesso para o bucket.")
#         except S3Error as err:
#             print(f"Erro ao enviar o arquivo {item}: {err}")

In [ ]:
# ## DADOS PARA INSERIR NA TABELA DE CONTROLE USANDO O BEAUTIFULSOAP##
# navegador = requests.get('https://www.openstreetmap.org/user/sunnypilot/traces/9255974')
# site = BeautifulSoup(navegador.text, 'html.parser')
# data_trace = site.find('div', attrs={'class': 'content-body'})
# table_trace = data_trace.find('table')
# 
# filename_row = table_trace.find('th', string='Filename:').parent
# owner_row = table_trace.find('th', string='Owner:').parent
# uploaded_row = table_trace.find('th', string='Uploaded:').parent
# 
# filename = filename_row.find('td').text.strip().replace('(download)','').strip()
# owner = owner_row.find('td').text.strip()
# uploaded = uploaded_row.find('td').text.strip()